# RLDT


## Step 1: Import the necessary libraries:


In [32]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

## Step 2: Define the environment:


### Step 2.1: Devices


#### _Gloabl variables_


In [33]:
num_IOT_devices = 10

voltages_frequencies_IOT = [
    (10e6  , 1.8),
    (20e6  , 2.3),
    (40e6  , 2.7),
    (80e6  , 4.0),
    (160e6 , 5.0),
]

num_MEC_devices = 5

voltages_frequencies_MEC = [
    (1500e6 ,  1.2),
    (1000e6 ,  1.0),
    (750e6, 0.825),
    (600e6, 0.8),
]

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5
num_dag_generations = 10000




#### _IOT_


In [34]:
devices_data_IOT = []
for i in range(num_IOT_devices):
    cpu_cores = np.random.choice([4, 8, 16])
    device_info = {
        "id": f"iot {i}",
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [0 for _ in range(cpu_cores)],
        "voltages_frequencies": [
            [
                voltages_frequencies_IOT[i]
                for i in np.random.choice(5, size=3, replace=False)
            ]
            for core in range(cpu_cores)
        ],
        "ISL": np.random.randint(10, 21) / 100,
        "capacitance": [np.random.uniform(2, 3) * 1e-10 for _ in range(cpu_cores)],
        "powerIdle": [
            np.random.choice([800, 900,1000]) * 1e-6 for _ in range(cpu_cores)
        ],
        "batteryLevel": np.random.randint(36, 41) * 1e9,
        "errorRate": np.random.randint(1, 6) / 100,
        "accetableTasks": list(np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        )),
        "handleSafeTask": np.random.choice([0, 1], p=[0.25, 0.75]),
    }
    devices_data_IOT.append(device_info)

IoTdevices = pd.DataFrame(devices_data_IOT)


#### _MEC_


In [35]:
devices_data_MEC = []
for i in range(num_MEC_devices):
    cpu_cores = np.random.choice([16, 32, 64])
    device_info = {
        "id":f"mec {i}",
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [0 for _ in range(cpu_cores)],
        "voltages_frequencies":[
            [
                voltages_frequencies_MEC[i]
                for i in np.random.choice(4, size=3, replace=False)
            ]
            for core in range(cpu_cores)
        ],
        "capacitance": [np.random.uniform(1.5, 2) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [np.random.choice([550, 650, 750]) * 1e-3 for _ in range(cpu_cores)],
        "errorRate": np.random.randint(5, 11) / 100,
        "accetableTasks": list(np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        )),
        "handleSafeTask": np.random.choice([0, 1], p=[0.75, 0.25]),
        "batteryLevel": 100,
        "ISL": 0,
    }
    devices_data_MEC.append(device_info)

MECDevices = pd.DataFrame(devices_data_MEC)


#### _CLOUD_


In [36]:
device_info = [
    {
        "id": 'cloud',
        "number_of_cpu_cores": 1,
        "occupied_cores": [0],
        "voltages_frequencies": [2.8e9, 3.9e9],
        "capacitance": (13.85, 24.28),
        "powerIdle": 0,
        "ISL": 0,
        "batteryLevel": 100,
        "errorRate": 0.1,
        "accetableTasks": [1, 2, 3, 4],
        "handleSafeTask": 0,
    }
]
cloud = pd.DataFrame(device_info)

#### ALL THE DEVICES


In [47]:
import ast

# devices = pd.concat([IoTdevices,MECDevices,cloud],ignore_index=True)
# devices.to_csv("devices.csv")

devices = pd.read_csv("devices.csv")
devices["voltages_frequencies"] = devices["voltages_frequencies"].apply(lambda x: ast.literal_eval(x))
devices["capacitance"] = devices["capacitance"].apply(
    lambda x: ast.literal_eval(x)
)
devices["powerIdle"] = devices["powerIdle"].apply(
    lambda x: ast.literal_eval(x)
)
devices["accetableTasks"] = devices["accetableTasks"].apply(
    lambda x: ast.literal_eval(x)
)
# devices = pd.concat([IoTdevices],ignore_index=True)

,Unnamed: 0,id,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask,acceptableTasks
0,0,iot 0,4,"[0, 0, 0, 0]","[[(160000000.0, 5.0), (10000000.0, 1.8), (8000...",0.12,"[2.785460842931057e-10, 2.6862015720275273e-10...","[0.0009, 0.0009, 0.0009, 0.001]",3.600000e+10,0.03,"[2, 1, 3]",1,"[2, 1, 3]"
1,1,iot 1,4,"[0, 0, 0, 0]","[[(40000000.0, 2.7), (20000000.0, 2.3), (16000...",0.16,"[2.8864332839389886e-10, 2.865145424918962e-10...","[0.0009, 0.0007999999999999999, 0.0009, 0.0009]",3.800000e+10,0.01,"[2, 1, 3, 4]",1,"[2, 1, 3, 4]"
2,2,iot 2,4,"[0, 0, 0, 0]","[[(80000000.0, 4.0), (160000000.0, 5.0), (4000...",0.12,"[2.766898727244802e-10, 2.5130996828859263e-10...","[0.001, 0.0009, 0.0009, 0.0007999999999999999]",3.700000e+10,0.01,"[3, 2, 1]",1,"[3, 2, 1]"
3,3,iot 3,8,"[0, 0, 0, 0, 0, 0, 0, 0]","[[(10000000.0, 1.8), (160000000.0, 5.0), (2000...",0.11,"[2.2681809706696604e-10, 2.5044559064937566e-1...","[0.0007999999999999999, 0.0009, 0.0009, 0.001,...",3.800000e+10,0.04,"[2, 4, 3]",0,"[2, 4, 3]"
4,4,iot 4,4,"[0, 0, 0, 0]","[[(10000000.0, 1.8), (20000000.0, 2.3), (16000...",0.12,"[2.074159677709666e-10, 2.934768517631425e-10,...","[0.001, 0.001, 0.0007999999999999999, 0.001]",3.700000e+10,0.02,"[2, 4, 3]",1,"[2, 4, 3]"
5,5,iot 5,8,"[0, 0, 0, 0, 0, 0, 0, 0]","[[(10000000.0, 1.8), (20000000.0, 2.3), (16000...",0.16,"[2.1523870654392426e-10, 2.0693294540528675e-1...","[0.001, 0.0009, 0.0007999999999999999, 0.00079...",3.600000e+10,0.05,"[4, 1, 2]",1,"[4, 1, 2]"
6,6,iot 6,4,"[0, 0, 0, 0]","[[(20000000.0, 2.3), (40000000.0, 2.7), (16000...",0.18,"[2.92162966178634e-10, 2.59608978778099e-10, 2...","[0.001, 0.0007999999999999999, 0.0007999999999...",3.600000e+10,0.05,"[2, 3, 4]",0,"[2, 3, 4]"
7,7,iot 7,8,"[0, 0, 0, 0, 0, 0, 0, 0]","[[(40000000.0, 2.7), (10000000.0, 1.8), (20000...",0.14,"[2.31680769871997e-10, 2.8452591847855804e-10,...","[0.0009, 0.001, 0.0007999999999999999, 0.001, ...",4.000000e+10,0.03,"[1, 3, 4, 2]",1,"[1, 3, 4, 2]"
8,8,iot 8,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(10000000.0, 1.8), (20000000.0, 2.3), (80000...",0.14,"[2.1837378361535037e-10, 2.0052774654974957e-1...","[0.0007999999999999999, 0.0009, 0.000799999999...",3.900000e+10,0.01,"[3, 4, 1]",1,"[3, 4, 1]"
9,9,iot 9,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[[(160000000.0, 5.0), (10000000.0, 1.8), (2000...",0.17,"[2.4516612062230635e-10, 2.1046315422958717e-1...","[0.001, 0.001, 0.0007999999999999999, 0.000799...",3.800000e+10,0.05,"[2, 4, 1, 3]",0,"[2, 4, 1, 3]"


### Step 2.2: Application


#### _helper function : generate_random_dag_


In [38]:
def generate_random_dag(num_nodes):
    dag = nx.DiGraph()

    nodes = [f"t{i+1}" for i in range(num_nodes)]
    dag.add_nodes_from(nodes)

    available_parents = {node: list(nodes[:i]) for i, node in enumerate(nodes)}

    for i in range(2, num_nodes + 1):

        num_parents = min(
            random.randint(1, min(i, max_num_parents_dag)), len(
                available_parents[f"t{i}"])
        )

        # select parents
        parent_nodes = random.sample(available_parents[f"t{i}"], num_parents)
        # add parents
        dag.add_edges_from((parent_node, f"t{i}")
                           for parent_node in parent_nodes)

        # update available parents
        available_parents[f"t{i}"] = list(nodes[:i])

    return dag

#### _Generate task DAGs_


In [39]:
tasks_data = []

start_node_number = 1
for run in range(num_dag_generations):

    num_nodes = random.randint(min_num_nodes_dag, max_num_nodes_dag)

    random_dag = generate_random_dag(num_nodes)

    mapping = {
        f"t{i}": f"t{i + start_node_number - 1}" for i in range(1, num_nodes + 1)
    }

    random_dag = nx.relabel_nodes(random_dag, mapping)
    for node in random_dag.nodes:
        parents = list(random_dag.predecessors(node))
        task_info = {
            "id": node,
            "job": run,
            "dependency": parents,
            "mobility": np.random.randint(1, 10),
            "kind": np.random.choice(task_kinds),
            "safe": np.random.choice([0, 1], p=[0.95, 0.05]),
            "computationalLoad": int(np.random.uniform(1, 11)*1e6),
            "dataEntrySize":int(np.random.uniform(1, 11)*1e6),
            "returnDataSize":int(np.random.uniform(1, 11)*1e6),
            "status": "READY",
        }
        tasks_data.append(task_info)
    start_node_number += num_nodes

np.random.shuffle(tasks_data)
tasks = pd.DataFrame(tasks_data)

tasks.set_index("id", inplace=True)
tasks

,job,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status
id,,,,,,,,,
t50971,4238,"[t50964, t50969]",5,4,0,4456113,1885887,4710625,READY
t12655,1055,"[t12650, t12651, t12652, t12653]",7,2,0,4387713,5646154,2138214,READY
t39095,3254,[t39094],4,4,1,2983077,4489805,9155933,READY
t109314,9056,"[t109312, t109313]",3,4,0,5476300,6103921,7580172,READY
t34072,2834,"[t34066, t34070]",9,4,0,4756749,9601424,10602616,READY
...,...,...,...,...,...,...,...,...,...
t81024,6714,"[t81019, t81020, t81022, t81023]",7,1,0,1172085,5036574,3606255,READY
t60133,4988,"[t60123, t60124, t60129, t60132]",2,3,0,2613579,4717671,10510798,READY
t63284,5236,[t63280],9,3,1,8238449,4654699,3277921,READY


## Step 4 : DDT


### Step 4.1: Initializing The tree


In [40]:
class DDT(nn.Module):
    def __init__(self, num_input, num_output, depth, max_depth):
        super(DDT, self).__init__()
        self.depth = depth
        self.max_depth = max_depth
        if depth != max_depth:
            # self.weights = nn.Parameter(torch.zeros(num_input))
            self.weights = nn.Parameter(torch.empty(
                num_input).normal_(mean=0, std=0.1))
            self.bias = nn.Parameter(torch.zeros(1))
        if depth == max_depth:
            self.prob_dist = nn.Parameter(torch.zeros(num_output))

        if depth < max_depth:
            self.left = DDT(num_input, num_output, depth + 1, max_depth)
            self.right = DDT(num_input, num_output, depth + 1, max_depth)

    def forward(self, x):
        if self.depth == self.max_depth:
            return self.prob_dist
        val = torch.sigmoid(torch.matmul(x, self.weights.t()) + self.bias)
        a = np.random.uniform(0, 1)
        if a < 0.1:
            val = 1 - val
        if val >= 0.5:

            return val * self.right(x)
        else:

            return (1 - val) * self.left(x)

## Step 5: RL


In [41]:
def calc_execution_time(device, task, core, dvfs):
    if device['id'] == "cloud":
        return task["computationalLoad"] / device["voltages_frequencies"][0]
    else:
        return task["computationalLoad"] / device["voltages_frequencies"][core][dvfs][0]


def calc_power_consumption(device, task, core, dvfs):
    if device['id'] == "cloud":
        return 13.85 * calc_execution_time(device, task, core, dvfs)
    return (
        device["capacitance"][core]
        * (device["voltages_frequencies"][core][dvfs][1] ** 2)
        * device["voltages_frequencies"][core][dvfs][0]
    )
def calc_energy(device, task, core, dvfs):
    return calc_execution_time(device, task, core, dvfs) * calc_power_consumption(device, task, core, dvfs)


def calc_total(device, task, core, dvfs):
    timeTransMec = 0
    timeTransCC = 0
    exeTime = 0
    e = 0

    transferRate5g =1e9
    latency5g=5e-3
    transferRateFiber =1e10
    latencyFiber=1e-3

    timeDownMec = task["returnDataSize"] / transferRate5g
    timeDownMec += latency5g
    timeUpMec = task["dataEntrySize"] / transferRate5g
    timeUpMec += latency5g

    alpha = 52e-5
    beta = 3.86412
    powerMec = alpha * 1e9 / 1e6 + beta

    timeDownCC = task["returnDataSize"] / transferRateFiber
    timeDownCC += latencyFiber
    timeUpCC = task["dataEntrySize"] / transferRateFiber
    timeUpCC += latencyFiber

    powerCC = 3.65 
    if device["id"].startswith("mec"):
        timeTransMec =  timeUpMec +  timeDownMec 
        energyTransMec = powerMec *  timeTransMec
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime = exeTime + timeTransMec 
        e = calc_energy(device, task, core, dvfs)
        totalEnergy =  e + energyTransMec

    elif device['id'].startswith("cloud"):
        timeTransMec =  timeUpMec +  timeDownMec 
        energyTransMec = powerMec * timeTransMec
        
        timeTransCC = timeUpCC+timeDownCC
        energyTransCC =  powerCC * timeTransCC
        
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime =  exeTime + timeTransMec +timeTransCC

        e = calc_energy(device, task, core, dvfs)
        totalEnergy =  + energyTransMec + energyTransCC

    elif device['id'].startswith("iot"):
        exeTime = calc_execution_time(device, task, core, dvfs)
        totalTime = exeTime
        e = calc_energy(device, task, core, dvfs)
        totalEnergy = e

    return totalTime , totalEnergy

In [42]:
# devices

In [43]:
tasks_copy = tasks.copy()
tasks_copy = tasks_copy.drop(["job","dependency","mobility","status"],axis=1)
taskList = tasks_copy.index.tolist()

In [44]:
def checkIfSuitable(state, device):
    punishment = 0
    safeFail = 0
    taskFail = 0
    if  state['safe'] and not device["handleSafeTask"]:
        punishment += 25
        safeFail += 1
        
    if state['kind'] not in device["accetableTasks"]:
        punishment += 25
        taskFail += 1
    
    return (20 if punishment > 0 else 0, taskFail, safeFail)

In [45]:
def getSetup(t, e, setup, alpha=1, beta=1):
    match setup:
        case "00":
            reward = -1 * (e + t)
        case "01":
            reward = -1 * (alpha * e + beta * t)
        case "02":
            reward = -1 / (e + t)
        case "03":
            reward = -1 / (alpha * e + beta * t)
        case "04":
            reward = -np.exp(e) - np.exp(t)
        case "05":
            reward = -np.exp(alpha * e) - np.exp(beta * t)
        case "06":
            reward = -np.exp(t + e)
        case "07":
            reward = -np.exp(alpha * t + beta * e)
        case "08":
            reward = np.exp(-t - e)
        case "09":
            reward = np.exp(-1 * (alpha * t + beta * e))
    
    return reward

In [46]:
class Environment:
    def __init__(self):
        self.totalSafeFail = 0
        self.totalTaskFail = 0
        self.totalReward = 0
        self.feature_size = 5
        self.num_actions = len(devices)
        self.max_depth = 3
        self.agent = DDT(self.feature_size, self.num_actions,
                         depth=0, max_depth=self.max_depth)
        self.optimizer = optim.Adam(self.agent.parameters(), lr=0.005)

    def execute_action(self, state, action):
        taskList.pop(0)
        device = devices.iloc[action]        

        punishment, taskFail, safeFail = checkIfSuitable(state, device)
        if safeFail:
            self.totalSafeFail += 1
        if taskFail:
            self.totalTaskFail += 1

        if not (punishment):
            for coreIndex in range(len(device["occupied_cores"])):
                if device["occupied_cores"][coreIndex] == 0:
                    total_t, total_e  = calc_total(device, state, coreIndex,np.random.randint(0,3))
                    reward = getSetup(total_t, total_e, "04")
                    # reward = -1 / (total_t + total_e)
                    # print(f"device {device['id']} ////// time {total_t}  ////// energy {total_e} ")
                    env.totalReward += reward
                    return (tasks_copy.loc[taskList[0]], reward, total_t, total_e)
        self.totalFail += 1
        return (tasks_copy.loc[taskList[0]], punishment, 0, 0)


    def train(self, num_epoch, num_episodes):

        total_avg_t = 0
        total_avg_e = 0
        total_avg_r = 0
        total_avg_l = 0

        for i in range(num_epoch):
            total_loss = 0
            env.totalFail = 0
            env.totalTaskFail = 0
            env.totalSafeFail = 0
            env.totalReward = 0
            total_loss = 0
            total_reward = 0
            totalTime = 0
            totalEnergy = 0


            
            for j in range(num_episodes):
                state = tasks_copy.loc[taskList[0]]
                x = torch.tensor(np.array(state.values, dtype=np.float32)).unsqueeze(0)
                
                output = self.agent(x)
                action_probabilities = torch.softmax(output, dim=0)
                action_index = torch.multinomial(action_probabilities, 1).item()

                next_state, reward, t, e = self.execute_action(state, action_index)
                loss = (output[action_index] * reward)

                total_reward += reward
                total_loss += loss
                totalTime += t
                totalEnergy += e
                
                

            self.optimizer.zero_grad()
            avg_loss = total_loss/num_episodes
            avg_time = totalTime / num_episodes
            avg_energy = totalEnergy / num_episodes
            
            # avg_reward = total_reward / num_episodes
            avg_reward = env.totalReward / num_episodes
            avg_loss = total_loss/num_episodes


            total_avg_t += avg_time
            total_avg_e += avg_energy
            total_avg_l += avg_loss
            total_avg_r += avg_reward
            

            avg_loss.backward()
            self.optimizer.step()
            if i % 100 == 0:
                # print(f"Epoch {i+1} // avg cc time: {avg_cc} // avg mec: {avg_mec} // avg og time: {avg_og} total fail: {env.totalFail} // Average Loss: {avg_loss}// ")
                # print(f"Epoch {i+1} // avg time: {avg_time} // avg added Time: {avg_added_time} // avg og time: {avg_og} total fail: {env.totalFail} // Average Loss: {avg_loss}// ")
                print(f"Epoch {i+1}  // safe/task fail: {env.totalSafeFail}/{env.totalTaskFail} // Average Loss: {avg_loss:.2f} // Total Reward: {env.totalReward:.2f} // Average Reward: {avg_reward:.2f} // Avg time: {avg_time:.2f} // Avg energy: {avg_energy:.2f}")
            
            # if i == 10001:
                
            #     print(f"safe/task fail: {env.totalSafeFail}/{env.totalTaskFail} // Average Loss: {avg_loss:.2f} // Total Reward: {env.totalReward:.2f} // Average Reward: {avg_reward:.2f} // Avg time: {avg_time:.2f} // Avg energy: {avg_energy:.2f}")

                # env.totalFail = 0
                env.totalSafeFail = 0
                env.totalTaskFail

        avg_avg_t = total_avg_t / num_epoch
        avg_avg_l = total_avg_l / num_epoch
        avg_avg_r = total_avg_r / num_epoch
        avg_avg_e = total_avg_e / num_epoch


        print("===============================================================================")
        print(f'Overall Average Time across all epochs: {avg_avg_t}')
        print(f'Overall Average e across all epochs: {avg_avg_e:.2f}')
        print(f'Overall Average l across all epochs: {avg_avg_l:.2f}')
        print(f'Overall Average r across all epochs: {avg_avg_r:.2f}')
     





# env.totalAddedAvg += avg_cc


env = Environment()
tree = env.agent
env.train(10001, 10)

print('///////////////////')

for name, param in env.agent.named_parameters():
    if "prob_dist" or "bias" not in name:
        # print(name,param)
        pass

TypeError: 'in <string>' requires string as left operand, not numpy.int64